Project Title : Customer Segmentation & Market Basket Analysis for Revenue Optimization

In [1]:
import pandas as pd 
import numpy as np
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt 
from itertools import combinations #is used to generate all possible item pairs inside one transaction (basket)
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
from collections import Counter #s used to count how many times each item and each pair occurs across all transactions, 
#which is needed to calculate pair frequency and confidence.

exported data from rfm segmentation file of segment wise so that we can connect each MBA rule to the correct customer segment, and then make targeted recommendations/actions instead of one common rule for everyone. 

In [2]:
df = pd.read_csv('merge data.csv')

In [3]:
df.head()

,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,TRANSACTION_ID,REVENUE,Segment_3D
0,2001-01-21,1069,Unknown,110333,4710320224661,1,361,425,TXN_1,425,Lost
1,2001-01-21,1069,Unknown,100311,4710022101208,1,197,198,TXN_1,198,Lost
2,2001-01-21,1069,Unknown,110333,4712603661644,1,313,348,TXN_1,348,Lost
3,2000-11-13,1069,Unknown,100205,9556439880610,1,80,89,TXN_2,89,Lost
4,2000-11-13,1069,Unknown,100314,4710176008699,1,78,98,TXN_2,98,Lost


why we extracted these 3 columns ??

Because these three columns let us link each transaction to the correct customer and their RFM segment, so we can analyze MBA rules segment-wise.

In [4]:
rfm_segments = df[['CUSTOMER_ID' ,'Segment_3D','TRANSACTION_ID']]
rfm_segments.head()


,CUSTOMER_ID,Segment_3D,TRANSACTION_ID
0,1069,Lost,TXN_1
1,1069,Lost,TXN_1
2,1069,Lost,TXN_1
3,1069,Lost,TXN_2
4,1069,Lost,TXN_2


In [5]:
basket_items = (
    df.groupby(["TRANSACTION_ID", "CUSTOMER_ID"])["PRODUCT_ID"]
      .apply(list)
      .reset_index()
)





In [6]:
basket_items.head()

,TRANSACTION_ID,CUSTOMER_ID,PRODUCT_ID
0,TXN_1,1069,"[4710320224661, 4710022101208, 4712603661644]"
1,TXN_10,1250,"[4719593555861, 4718585391258, 4905660070157, ..."
2,TXN_100,15424,"[9556001804068, 4718433613228, 4710527519102, ..."
3,TXN_1000,33510,"[4710757025602, 4710011402019, 4719090106009, ..."
4,TXN_10000,289375,[4710908110232]


We do this to combine all products bought in the same transaction (by the same customer) into one basket, so MBA can learn which items were purchased together.

analyzing champion customer  data only 

In [7]:
champions_df = df[df['Segment_3D'] == 'Champions']
champions_df.head()


,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,TRANSACTION_ID,REVENUE,Segment_3D
220,2001-01-16,5241,35-39,100507,4710626564966,1,63,85,TXN_29,85,Champions
221,2001-01-16,5241,35-39,110411,4710088412126,1,24,30,TXN_29,30,Champions
222,2001-01-16,5241,35-39,100422,4710967900232,1,49,60,TXN_29,60,Champions
223,2001-01-16,5241,35-39,530209,4710731060124,2,80,110,TXN_29,220,Champions
224,2001-01-16,5241,35-39,110411,4710110241175,1,31,39,TXN_29,39,Champions


analyzing Loyal Regulars customer data only

In [8]:
loyal_df = df[df['Segment_3D'] == 'Loyal Regulars']
loyal_df.head()


,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,TRANSACTION_ID,REVENUE,Segment_3D
29,2001-02-10,1250,35-39,100109,4710015104841,1,74,89,TXN_9,89,Loyal Regulars
30,2001-02-10,1250,35-39,100205,4710176001829,1,51,65,TXN_9,65,Loyal Regulars
31,2001-02-10,1250,35-39,100205,4710176001812,1,51,65,TXN_9,65,Loyal Regulars
32,2001-02-10,1250,35-39,100102,723125488026,1,20,65,TXN_9,65,Loyal Regulars
33,2001-02-10,1250,35-39,590522,20480349,1,302,395,TXN_9,395,Loyal Regulars


analyzing at risk customer data only

In [9]:
at_risk_df = df[df['Segment_3D'] == 'At Risk Regulars']
at_risk_df.head()


,TRANSACTION_DT,CUSTOMER_ID,AGE_GROUP,PRODUCT_SUBCLASS,PRODUCT_ID,AMOUNT,ASSET,SALES_PRICE,TRANSACTION_ID,REVENUE,Segment_3D
11,2001-01-06,1113,Unknown,110605,4710254015014,2,68,84,TXN_5,168,At Risk Regulars
12,2001-01-06,1113,Unknown,110136,4710008251125,1,23,28,TXN_5,28,At Risk Regulars
13,2001-01-06,1113,Unknown,110605,4710254015021,2,68,84,TXN_5,168,At Risk Regulars
14,2001-01-06,1113,Unknown,100312,37000440147,1,40,47,TXN_5,47,At Risk Regulars
15,2001-01-06,1113,Unknown,110401,4901734003182,1,135,169,TXN_5,169,At Risk Regulars


In [10]:


def mba_pair_rules(data, transaction_col, item_col,
                   min_pair_count=30, min_confidence=0.2, top_n=20):

    # 1) Group items by transaction
    txns = data.groupby(transaction_col)[item_col].apply(lambda x: set(x)).tolist()

    # 2) Count item frequency + pair frequency
    item_counts = Counter()
    pair_counts = Counter()

    for items in txns:
        item_counts.update(items)

        # count all pairs
        for a, b in combinations(sorted(items), 2):
            pair_counts[(a, b)] += 1

    # 3) Convert to rules
    rules = []

    for (a, b), pair_count in pair_counts.items():

        if pair_count < min_pair_count:
            continue

        # confidence A -> B
        conf_a_b = pair_count / item_counts[a]
        # confidence B -> A
        conf_b_a = pair_count / item_counts[b]

        if conf_a_b >= min_confidence:
            rules.append((a, b, pair_count, conf_a_b))

        if conf_b_a >= min_confidence:
            rules.append((b, a, pair_count, conf_b_a))

    rules_df = pd.DataFrame(rules, columns=["antecedent", "consequent", "pair_count", "confidence"])

    # sort by confidence first then count
    rules_df = rules_df.sort_values(["confidence", "pair_count"], ascending=False)

    return rules_df.head(top_n)


this function does MBA in a smarter way:

It directly counts item pairs inside transactions, and creates rules from them.

pair counting inside transactions (much lighter)

It helps you find:

frequently bought together pairs

cross-sell opportunities

bundle candidates

recommendation pairs

shelf placement pairs

Due to memory limitations, I used a pair-based MBA approach which is still highly useful for business because most recommendations are 2-item cross-sells.”

In [11]:
champions_data = df[df["Segment_3D"] == "Champions"]

champions_rules = mba_pair_rules(
    champions_data,
    transaction_col="TRANSACTION_ID",
    item_col="PRODUCT_ID",
    min_pair_count=30,
    min_confidence=0.2,
    top_n=20
)

champions_rules


,antecedent,consequent,pair_count,confidence
215,93447690,93432634,39,0.847826
286,4710175567173,4710175567166,57,0.802817
85,4719090790017,4719090790000,177,0.797297
234,93447690,93362979,36,0.782609
479,93432641,93432634,39,0.780000
219,93457545,93432634,35,0.760870
222,93447690,93457545,35,0.760870
223,93457545,93447690,35,0.760870
211,93447706,93362993,37,0.755102
369,4718752200024,4718752200017,46,0.754098


We do this to run Market Basket Analysis only on “Champions” customers, so we get product-pair rules specific to high-value customers, which helps in making targeted recommendations, bundles, and offers for that segment instead of using one common rule for everyone.

MBA rules were further analyzed by RFM customer segments to understand segment-specific purchasing behavior. This helped identify which product associations are driven by high-value customers and which are more common among low-frequency or at-risk customers. Segment-wise rules were mapped to business actions such as targeted recommendations, personalized bundling, and retention campaigns.

segment wise these product purchese analyze . how many time which segment customer purchased 

In [12]:
item1 = "4710011401128"
item2 = "4710011401135"

temp = df[df["PRODUCT_ID"].astype(str).isin([item1, item2])]

pairs = (
    temp.groupby(["Segment_3D", "TRANSACTION_ID"])["PRODUCT_ID"]
    .nunique()
    .reset_index()
)

pairs = pairs[pairs["PRODUCT_ID"] == 2]

pair_count_by_segment = pairs.groupby("Segment_3D")["TRANSACTION_ID"].nunique()

print(pair_count_by_segment)


Segment_3D
At Risk Regulars     53
At Risk Whales       53
Champions           374
Lost                113
Loyal Regulars      104
New Customers         3
Name: TRANSACTION_ID, dtype: int64


no purchase of these combo

In [13]:
item1 = "{4710011401128}"
item2 = "{4710011401133}"

temp = df[df["PRODUCT_ID"].astype(str).isin([item1, item2])]

pairs = (
    temp.groupby(["Segment_3D", "TRANSACTION_ID"])["PRODUCT_ID"]
    .nunique()
    .reset_index()
)

# Keep only those transactions where both items exist
pairs = pairs[pairs["PRODUCT_ID"] == 2]

# Count how many such transactions happened in each segment
pair_count_by_segment = pairs.groupby("Segment_3D")["TRANSACTION_ID"].nunique()

print(pair_count_by_segment)


Series([], Name: TRANSACTION_ID, dtype: int64)


In [14]:
temp = df[df["PRODUCT_ID"].astype(str).isin([item1, item2])]

check = temp.groupby("TRANSACTION_ID")["PRODUCT_ID"].nunique().value_counts()
print(check)


Series([], Name: count, dtype: int64)


In [15]:


itemA = "4710011401128"
itemB = "4710011401135"

# Keep only needed columns
data = df[["Segment_3D", "TRANSACTION_ID", "PRODUCT_ID"]].copy()
data["PRODUCT_ID"] = data["PRODUCT_ID"].astype(str)

# Function to calculate metrics per segment
results = []

for seg, seg_df in data.groupby("Segment_3D"):

    # Total transactions in this segment
    total_txn = seg_df["TRANSACTION_ID"].nunique()

    # Transactions that contain item A
    txn_A = seg_df[seg_df["PRODUCT_ID"] == itemA]["TRANSACTION_ID"].nunique()

    # Transactions that contain item B
    txn_B = seg_df[seg_df["PRODUCT_ID"] == itemB]["TRANSACTION_ID"].nunique()

    # Transactions that contain BOTH A and B
    temp = seg_df[seg_df["PRODUCT_ID"].isin([itemA, itemB])]

    txn_AB = (
        temp.groupby("TRANSACTION_ID")["PRODUCT_ID"]
        .nunique()
        .reset_index()
    )

    txn_AB = txn_AB[txn_AB["PRODUCT_ID"] == 2]["TRANSACTION_ID"].nunique()

    # Support
    support_A = txn_A / total_txn if total_txn else 0
    support_B = txn_B / total_txn if total_txn else 0
    support_AB = txn_AB / total_txn if total_txn else 0

    # Confidence
    conf_A_to_B = txn_AB / txn_A if txn_A else 0
    conf_B_to_A = txn_AB / txn_B if txn_B else 0

    # Lift
    lift_A_to_B = support_AB / (support_A * support_B) if support_A * support_B else 0

    results.append({
        "Segment": seg,
        "Total_Transactions": total_txn,
        "Txn_A": txn_A,
        "Txn_B": txn_B,
        "Txn_AB": txn_AB,
        "Support_AB": support_AB,
        "Confidence_A->B": conf_A_to_B,
        "Confidence_B->A": conf_B_to_A,
        "Lift": lift_A_to_B
    })

rules_segmentwise = pd.DataFrame(results)

rules_segmentwise


,Segment,Total_Transactions,Txn_A,Txn_B,Txn_AB,Support_AB,Confidence_A->B,Confidence_B->A,Lift
0,At Risk Regulars,5683,103,68,53,0.009326,0.514563,0.779412,43.003855
1,At Risk Whales,7502,109,73,53,0.007065,0.486239,0.726027,49.969335
2,Big Spenders,912,2,0,0,0.000000,0.000000,0.000000,0.000000
3,Champions,58014,923,511,374,0.006447,0.405200,0.731898,46.002540
4,Lost,24409,253,144,113,0.004629,0.446640,0.784722,75.708635
5,Loyal Regulars,17561,240,130,104,0.005922,0.433333,0.800000,58.536667
6,New Customers,5497,7,4,3,0.000546,0.428571,0.750000,588.964286


This code tells you which customer segment actually drives this rule, and in which segment this pair is strongest

we do this code because we want to check how strong the SAME rule (A ↔ B) is inside each RFM segment, instead of only seeing the overall dataset result.

If lift is high in Champions:

✅ show recommendation to Champions
✅ bundle offer for Champions
✅ keep products near each other for premium customers

# kpi mapping segement wise

CHAMPIONS

In [16]:
champions = df[df["Segment_3D"] == "Champions"]

rev_per_customer = champions.groupby("CUSTOMER_ID")["REVENUE"].sum().mean()
print("Champions Revenue per Customer:", rev_per_customer)


Champions Revenue per Customer: 87154.8022341095


In [17]:
basket_value = champions.groupby("TRANSACTION_ID")["REVENUE"].sum().mean()
print("Champions Basket Value:", basket_value)


Champions Basket Value: 9548.65934084876


% of customers who buy a higher-priced / premium item when it is recommended.

In [18]:
upsell_rate = (
    champions.groupby("TRANSACTION_ID")["PRODUCT_ID"]
    .nunique()
    .gt(1)
    .mean()
)

print("Champions Upsell Success Rate:", upsell_rate)


Champions Upsell Success Rate: 0.875995449374289


In [19]:
at_risk = df[df["Segment_3D"] == "At Risk"].copy()

# Count unique transactions per customer
risk_counts = (
    at_risk[["CUSTOMER_ID", "TRANSACTION_ID"]]
    .drop_duplicates()
    .groupby("CUSTOMER_ID")["TRANSACTION_ID"]
    .nunique()
)

# Repeat purchase rate (proxy retention)
retention_rate = (risk_counts >= 2).mean()

print("At Risk Repeat Purchase Rate (>=2 txns):", retention_rate)


At Risk Repeat Purchase Rate (>=2 txns): nan


eed purchase dates and a time window

NEW CUSTOMER

% of new customers who make a purchase (or a second purchase) after seeing a recommendation/offer

In [20]:
new_cust = df[df["Segment_3D"] == "New Customers"]

purchase_counts = new_cust.groupby("CUSTOMER_ID")["TRANSACTION_ID"].nunique()

conversion_rate = (purchase_counts >= 2).mean()
print("New Customer Conversion Rate:", conversion_rate)


New Customer Conversion Rate: 0.4945092656142759


Final Output Table Segment KPI

In [21]:


# Segment KPIs
segment_kpi = df.groupby("Segment_3D").agg(
    customers=("CUSTOMER_ID", "nunique"),
    transactions=("TRANSACTION_ID", "nunique"),
    revenue=("REVENUE", "sum"),
)

# Revenue KPIs
segment_kpi["revenue_per_customer"] = segment_kpi["revenue"] / segment_kpi["customers"]
segment_kpi["basket_value"] = segment_kpi["revenue"] / segment_kpi["transactions"]

# Repeat Rate (>= 2 transactions per customer)
repeat_rate = (
    df[["Segment_3D", "CUSTOMER_ID", "TRANSACTION_ID"]]
    .drop_duplicates()
    .groupby(["Segment_3D", "CUSTOMER_ID"])["TRANSACTION_ID"]
    .nunique()
    .reset_index(name="txn_count")
    .groupby("Segment_3D")["txn_count"]
    .apply(lambda x: (x >= 2).mean())
)

segment_kpi["repeat_rate"] = repeat_rate

print(segment_kpi)


                  customers  transactions    revenue  revenue_per_customer  \
Segment_3D                                                                   
At Risk Regulars       2972          5683    6296781           2118.701548   
At Risk Whales         3045          7502   72618013          23848.280131   
Big Spenders            360           912    8381951          23283.197222   
Champions              6356         58014  553955923          87154.802234   
Lost                  11861         24409  157037728          13239.838799   
Loyal Regulars         4758         17561   11000774           2312.058428   
New Customers          2914          5497    3041638           1043.801647   

                  basket_value  repeat_rate  
Segment_3D                                   
At Risk Regulars   1108.002991     0.497981  
At Risk Whales     9679.820448     0.640066  
Big Spenders       9190.735746     0.708333  
Champions          9548.659341     0.989773  
Lost               64

Repeat Rate = % customers who made 2 or more transactions in that segment.

In [22]:


itemA = "4710011401128"
itemB = "4710011401135"

# Keep only required columns
data = df[["Segment_3D", "TRANSACTION_ID", "PRODUCT_ID"]].copy()
data["PRODUCT_ID"] = data["PRODUCT_ID"].astype(str)

results = []

for seg, seg_df in data.groupby("Segment_3D"):

    # total unique transactions in this segment
    total_txn = seg_df["TRANSACTION_ID"].nunique()

    # transactions containing A
    txn_A = seg_df[seg_df["PRODUCT_ID"] == itemA]["TRANSACTION_ID"].nunique()

    # transactions containing B
    txn_B = seg_df[seg_df["PRODUCT_ID"] == itemB]["TRANSACTION_ID"].nunique()

    # transactions containing both A and B
    temp = seg_df[seg_df["PRODUCT_ID"].isin([itemA, itemB])]
    
    txn_AB = (
        temp.groupby("TRANSACTION_ID")["PRODUCT_ID"]
        .nunique()
        .reset_index()
    )
    
    txn_AB = txn_AB[txn_AB["PRODUCT_ID"] == 2]["TRANSACTION_ID"].nunique()

    # Supports
    support_A = txn_A / total_txn if total_txn else 0
    support_B = txn_B / total_txn if total_txn else 0
    support_AB = txn_AB / total_txn if total_txn else 0

    # A -> B
    confidence_A_B = txn_AB / txn_A if txn_A else 0
    lift_A_B = support_AB / (support_A * support_B) if support_A * support_B else 0

    # B -> A
    confidence_B_A = txn_AB / txn_B if txn_B else 0
    lift_B_A = support_AB / (support_A * support_B) if support_A * support_B else 0

    # Save both directional rules
    results.append({
        "Segment": seg,
        "Antecedent": itemA,
        "Consequent": itemB,
        "Support": support_AB,
        "Confidence": confidence_A_B,
        "Lift": lift_A_B
    })

    results.append({
        "Segment": seg,
        "Antecedent": itemB,
        "Consequent": itemA,
        "Support": support_AB,
        "Confidence": confidence_B_A,
        "Lift": lift_B_A
    })

rules_segmentwise = pd.DataFrame(results)

# Sort: high lift first
rules_segmentwise = rules_segmentwise.sort_values(["Segment", "Lift"], ascending=[True, False])

rules_segmentwise


,Segment,Antecedent,Consequent,Support,Confidence,Lift
0,At Risk Regulars,4710011401128,4710011401135,0.009326,0.514563,43.003855
1,At Risk Regulars,4710011401135,4710011401128,0.009326,0.779412,43.003855
2,At Risk Whales,4710011401128,4710011401135,0.007065,0.486239,49.969335
3,At Risk Whales,4710011401135,4710011401128,0.007065,0.726027,49.969335
4,Big Spenders,4710011401128,4710011401135,0.000000,0.000000,0.000000
5,Big Spenders,4710011401135,4710011401128,0.000000,0.000000,0.000000
6,Champions,4710011401128,4710011401135,0.006447,0.405200,46.002540
7,Champions,4710011401135,4710011401128,0.006447,0.731898,46.002540
8,Lost,4710011401128,4710011401135,0.004629,0.446640,75.708635
9,Lost,4710011401135,4710011401128,0.004629,0.784722,75.708635


segment wise for item A & item B 

# ACTION PLAN TABLE 

In [23]:


# Create action plan mapping
action_plan = {
    "Champions": {
        "Goal": "Maximize Customer Lifetime Value",
        "Strategy": "Premium bundles & early access offers",
        "Channel": "App / Email",
        "KPI": "Increase Basket Value"
    },
    "Big Spenders": {
        "Goal": "Retain & Upsell",
        "Strategy": "Exclusive loyalty tier benefits",
        "Channel": "App",
        "KPI": "Increase Repeat Rate"
    },
    "Loyal Regulars": {
        "Goal": "Increase Average Basket Value",
        "Strategy": "Combo offers & add-on recommendations",
        "Channel": "Online + Store",
        "KPI": "Basket Value Growth"
    },
    "New Customers": {
        "Goal": "Improve Second Purchase Rate",
        "Strategy": "7-day repeat discount incentive",
        "Channel": "SMS / App Push",
        "KPI": "Repeat Rate"
    },
    "At Risk Whales": {
        "Goal": "Urgent Reactivation",
        "Strategy": "Personalized high-value win-back offer",
        "Channel": "Email + Call",
        "KPI": "Reactivation Rate"
    },
    "At Risk Regulars": {
        "Goal": "Prevent Churn",
        "Strategy": "Limited-time targeted discount",
        "Channel": "SMS",
        "KPI": "Recency Reduction"
    },
    "Lost": {
        "Goal": "Selective Recovery",
        "Strategy": "Cashback comeback offer",
        "Channel": "Email",
        "KPI": "Recovery ROI"
    }
}

# Convert to DataFrame
action_df = pd.DataFrame(action_plan).T.reset_index()
action_df.rename(columns={"index": "Segment"}, inplace=True)




In [24]:
action_df.head()

,Segment,Goal,Strategy,Channel,KPI
0,Champions,Maximize Customer Lifetime Value,Premium bundles & early access offers,App / Email,Increase Basket Value
1,Big Spenders,Retain & Upsell,Exclusive loyalty tier benefits,App,Increase Repeat Rate
2,Loyal Regulars,Increase Average Basket Value,Combo offers & add-on recommendations,Online + Store,Basket Value Growth
3,New Customers,Improve Second Purchase Rate,7-day repeat discount incentive,SMS / App Push,Repeat Rate
4,At Risk Whales,Urgent Reactivation,Personalized high-value win-back offer,Email + Call,Reactivation Rate


In [25]:
action_df.tail()

,Segment,Goal,Strategy,Channel,KPI
2,Loyal Regulars,Increase Average Basket Value,Combo offers & add-on recommendations,Online + Store,Basket Value Growth
3,New Customers,Improve Second Purchase Rate,7-day repeat discount incentive,SMS / App Push,Repeat Rate
4,At Risk Whales,Urgent Reactivation,Personalized high-value win-back offer,Email + Call,Reactivation Rate
5,At Risk Regulars,Prevent Churn,Limited-time targeted discount,SMS,Recency Reduction
6,Lost,Selective Recovery,Cashback comeback offer,Email,Recovery ROI


In [26]:
df.to_csv("action_df.csv" , index = False)

In [27]:
df.to_csv("segment_kpi.csv" , index = False)

“I identified 7 behavioral segments and designed targeted revenue optimization strategies based on basket value and repeat rate patterns.”

# Proposed Validation Framework

As this study is academic in nature, real-world campaign execution was not performed. However, the effectiveness of the proposed strategies can be evaluated through:

Scenario-based revenue simulations

KPI sensitivity analysis

A structured A/B testing framework for future implementation

Projected revenue uplift was estimated by modeling improvements in basket value, repeat rate, and reactivation rate under conservative (5%), moderate (10%), and aggressive (15%) growth scenarios.

This approach ensures the strategic recommendations are quantitatively justifiable even without real deployment data.

# Final Conclusion

This project demonstrates how combining RFM segmentation with Market Basket Analysis creates a powerful framework for targeted recommendations and revenue growth.

Instead of applying one universal strategy, the analysis enables segment-driven decision-making, ensuring that each customer group receives the most effective intervention.

The outcome is a scalable and data-driven approach to improve revenue, retention, and overall customer lifetime value.